In [1]:
import pandas as pd

In [33]:
data = pd.read_csv('wiki_data.txt', names=['text'], sep='\t', on_bad_lines='skip')

In [35]:
data.shape

(20002, 1)

# Задание 1 (3 балла)

Обучите word2vec модели с негативным семплированием (cbow и skip-gram) с помощью tensorflow аналогично тому, как это было сделано в семинаре. Вам нужно изменить следующие пункты: 
1) добавьте лемматизацию в предобработку (любым способом)  
2) измените размер окна на 6 для cbow и 12 для skip gram (обратите внимание, что размер окна = #слов слева + #слов справа, в gen_batches в семинаре window используется не так и вам нужно это  изменить!)  

Выберете несколько не похожих по смыслу слов, и протестируйте полученные эмбединги (найдите ближайшие слова и оцените правильность, как в семинаре)

**Лемматизируем тексты**

In [6]:
import pymorphy2
from nltk.tokenize import wordpunct_tokenize

morph = pymorphy2.MorphAnalyzer(lang='ru')
storage = {}

def lemma_generator(tokens: list) -> list:
    global storage
    lemma = ''
    for tok in tokens:
        if tok.isalpha():
            if not tok in storage:
                lemma = morph.parse(tok)[0].normal_form
                storage[tok] = lemma
            else:
                lemma = storage[tok]
            yield lemma

def text_to_lemma(s: str) -> str:
    global morph
    tokens = wordpunct_tokenize(s)
    return ' '.join(list(lemma_generator(tokens)))

In [36]:
%%time
data['lemma'] = data.text.apply(text_to_lemma)

Wall time: 2min 1s


In [37]:
data.lemma.head()

0    новостройка нижегородский область новострый йк...
1    эсмеральда фильм эсмеральда немой короткометра...
2    список остров архипелаг норденшёльд это рабочи...
3    минкин ми нкина ми нкина фамилия получить расп...
4    пероральный приём лекарственный средство перор...
Name: lemma, dtype: object

**Подготовка данных**

In [106]:
from collections import Counter

vocab = Counter()

for text in data.lemma:
    vocab.update(text.split())
    
filtered_vocab = set([word for word in vocab if vocab[word] > 30])
word2id = {word: i + 1 for i, word in enumerate(filtered_vocab)}
word2id['PAD'] = 0
id2word = {i: word for word, i in word2id.items()}
        
def text_in_idx_generator(texts):
    global word2id
    for l in texts:
        yield [word2id[token] for token in l if token in word2id]
        
sentences = list(text_in_idx_generator(data.lemma.apply(str.split)))

**Вспомогательные функции**

In [59]:
import tensorflow as tf
import numpy as np

# skip gram
def gen_batches_sg(sentences, window = 5, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window):i] + sent[i+1:i+window]
                for context_word in context:
                    X_target.append(word)
                    X_context.append(context_word)
                    y.append(1)
                    
                    X_target.append(word)
                    X_context.append(np.random.randint(vocab_size))
                    y.append(0)
                    
                    if len(X_target) >= batch_size:
                        X_target = np.array(X_target)
                        X_context = np.array(X_context)
                        y = np.array(y)
                        yield ((X_target, X_context), y)
                        X_target = []
                        X_context = []
                        y = []

# # cbow 
def gen_batches_cbow(sentences, window = 5, batch_size=1000):
    while True:
        X_target = []
        X_context = []
        y = []

        for sent in sentences:
            for i in range(len(sent)-1):
                word = sent[i]
                context = sent[max(0, i-window):i] + sent[i+1:i+window]

                X_target.append(word)
                X_context.append(context)
                y.append(1)
                
                X_target.append(np.random.randint(vocab_size))
                X_context.append(context)
                y.append(0)

                if len(X_target) == batch_size:
                    X_target = np.array(X_target)
                    X_context = tf.keras.preprocessing.sequence.pad_sequences(X_context, maxlen=window*2)
                    y = np.array(y)
                    yield ((X_target, X_context), y)
                    X_target = []
                    X_context = []
                    y = []

In [69]:
vocab_size = len(id2word)

**CBOW**

In [68]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(10,))

embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Lambda(lambda x: tf.keras.backend.sum(x, axis=1))(embeddings_context)
dot = tf.keras.layers.Dot(1)([target, context])

outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)


optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [72]:
model.fit(gen_batches_cbow(sentences[:19000], window=6),
          validation_data=gen_batches_cbow(sentences[19000:],  window=6),
          batch_size=1000,
          steps_per_epoch=2500,
          validation_steps=30,
          epochs=3)

Epoch 1/3
2500/2500 [==============================] - 250s 100ms/step - loss: 0.2694 - accuracy: 0.8876 - val_loss: 0.3000 - val_accuracy: 0.8775
Epoch 2/3
2500/2500 [==============================] - 249s 100ms/step - loss: 0.2917 - accuracy: 0.8812 - val_loss: 0.2692 - val_accuracy: 0.8914
Epoch 3/3
2500/2500 [==============================] - 248s 99ms/step - loss: 0.2725 - accuracy: 0.8902 - val_loss: 0.2750 - val_accuracy: 0.8894


**Skip-gram**

In [73]:
inputs_target = tf.keras.layers.Input(shape=(1,))
inputs_context = tf.keras.layers.Input(shape=(1,))

embeddings_target = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_target, )
embeddings_context = tf.keras.layers.Embedding(input_dim=len(word2id), output_dim=300)(inputs_context, )

target = tf.keras.layers.Flatten()(embeddings_target)
context = tf.keras.layers.Flatten()(embeddings_context)

dot = tf.keras.layers.Dot(1)([target, context])
outputs = tf.keras.layers.Activation(activation='sigmoid')(dot)

model_sg = tf.keras.Model(inputs=[inputs_target, inputs_context], 
                       outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model_sg.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [74]:
model_sg.fit(gen_batches_sg(sentences[:19000], window=12),
             validation_data=gen_batches_sg(sentences[19000:],  window=12),
             batch_size=1000,
             steps_per_epoch=2500,
             validation_steps=30,
             epochs=3)

Epoch 1/3
2500/2500 [==============================] - 232s 92ms/step - loss: 0.5016 - accuracy: 0.7565 - val_loss: 0.5265 - val_accuracy: 0.7468
Epoch 2/3
2500/2500 [==============================] - 229s 92ms/step - loss: 0.4555 - accuracy: 0.7922 - val_loss: 0.5057 - val_accuracy: 0.7884
Epoch 3/3
2500/2500 [==============================] - 232s 93ms/step - loss: 0.4577 - accuracy: 0.7959 - val_loss: 0.5718 - val_accuracy: 0.7539


**Оценка эмбеддингов**

In [78]:
from sklearn.metrics.pairwise import cosine_distances

def most_similar(word, cbow, skipgram, n=5):
    global id2word, word2id
    similar_cbow = [id2word[i] for i in 
                    cosine_distances(cbow[word2id[word]].reshape(1, -1), cbow).argsort()[0][:n]]
    similar_sg = [id2word[i] for i in 
                    cosine_distances(skipgram[word2id[word]].reshape(1, -1), skipgram).argsort()[0][:n]]
    print(f"{word}\nCBOW-similar: {similar_cbow}\nSkip-gram: {similar_sg}")

In [79]:
embeddings_cbow = model.layers[2].get_weights()[0]
embeddings_sg = model_sg.layers[2].get_weights()[0]

In [80]:
most_similar('университет', embeddings_cbow, embeddings_sg, n=10)

университет
CBOW-similar: ['университет', 'институт', 'студент', 'школа', 'академия', 'факультет', 'научный', 'училище', 'кафедра', 'колледж']
Skip-gram: ['университет', 'нарвский', 'курс', 'кафедра', 'петроградский', 'вступить', 'декабрь', 'приезжать', 'братский', 'руководитель']


In [81]:
most_similar('красный', embeddings_cbow, embeddings_sg, n=10)

красный
CBOW-similar: ['красный', 'белый', 'зелёный', 'чёрный', 'цвет', 'большой', 'сторона', 'орёл', 'и', 'звезда']
Skip-gram: ['красный', 'хлопин', 'действительно', 'мёртвый', 'солнце', 'семён', 'раствор', 'факт', 'оружие', 'отсчёт']


In [82]:
most_similar('женщина', embeddings_cbow, embeddings_sg, n=10)

женщина
CBOW-similar: ['женщина', 'мужчина', 'женский', 'ребёнок', 'девушка', 'одежда', 'костюм', 'лицо', 'трое', 'картина']
Skip-gram: ['женщина', 'швеция', 'соревнование', 'лос', 'пробиться', 'закончиться', 'страна', 'интервью', 'парусный', 'долгосрочный']


Как видно, CBOW (также имея в виду его меньшее контекстное окно) предсказывает слова, которые могли бы заместить целевое слово синтаксически (семантически слова близки лишь отчасти, поскольку относятся к одной группе вроде *цвета* или *люди*)

    университет = институт
    красный = белый = зелёный = чёрный 
    женщина = мужчина = ребёнок = девушка
    
Тогда как Skip-gram показывает слова, которые находятся рядом с целевым словом, что хорошо видно на примере слова *университет*

# Задание 2 (3 балла)

Обучите 1 word2vec и 1 fastext модель в gensim. В каждой из модели нужно задать все параметры, которые мы разбирали на семинаре. Заданные значения должны отличаться от дефолтных и от тех, что мы использовали на семинаре.

In [11]:
import gensim

In [38]:
%%time
# CBOW with negative sampling
w2v = gensim.models.Word2Vec(data.lemma.apply(str.split), 
                             vector_size=100, 
                             min_count=30, 
                             max_vocab_size=20000,
                             window=6,
                             epochs=10,
                             hs=0,
                             negative=10)

Wall time: 1min 2s


In [39]:
w2v.wv.most_similar('утро')

[('утром', 0.7718522548675537),
 ('ночь', 0.7375233769416809),
 ('вечером', 0.7304009795188904),
 ('час', 0.7204369306564331),
 ('вечер', 0.6180065870285034),
 ('ночью', 0.6135960221290588),
 ('день', 0.608790934085846),
 ('минута', 0.5820714235305786),
 ('сутки', 0.5688872337341309),
 ('подступ', 0.5477177500724792)]

In [40]:
%%time
ft = gensim.models.FastText(data.lemma.apply(str.split), 
                            min_n=3, 
                            max_n=9)

Wall time: 4min 31s


In [41]:
ft.wv.most_similar('утро')

[('утром', 0.8683101534843445),
 ('дутро', 0.84256911277771),
 ('скороход', 0.6741518974304199),
 ('оса', 0.657135009765625),
 ('рм', 0.6513649225234985),
 ('лёд', 0.641215980052948),
 ('боезапас', 0.6369556784629822),
 ('дымоход', 0.6338866949081421),
 ('снегоход', 0.6317901611328125),
 ('плацдарм', 0.6232016682624817)]

# Задание 3 (4 балла)

Используя датасет для классификации (labeled.csv) и простую нейронную сеть (последняя модель в семинаре), оцените качество полученных эмбедингов в задании 1 и 2 (4 набора эмбедингов), также проверьте 1 любую из предобученных моделей с rus-vectores (но только не tayga_upos_skipgram_300_2_2019). 
Какая модель показывает наилучший результат?

Убедитесь, что для каждой модели вы корректно воспроизводите пайплайн предобработки (в 1 задании у вас лемматизация, не забудьте ее применить к датасету для классификации; у выбранной предобученной модели может быть своя специфичная предобработка - ее нужно воспроизвести)

In [83]:
toxic = pd.read_csv("labeled.csv")

In [84]:
toxic['lemma'] = toxic.comment.apply(text_to_lemma)

**Обрабатываем лемматизированные данные**

In [107]:
vocab = Counter()

for text in toxic.lemma:
    vocab.update(text.split())
    
filtered_vocab = set([word for word in vocab if vocab[word] > 30])
word2id_1 = {word: i + 1 for i, word in enumerate(filtered_vocab)}
word2id_1['PAD'] = 0
id2word_1 = {i: word for word, i in word2id.items()}
        
X = list(text_in_idx_generator(toxic.lemma.apply(str.split)))
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=100)
y = toxic.toxic.values

**Разбиваем их на обучающую и тестовую выборки**

In [91]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

**Учим модели, основанные на Word2Vec & FastText предобученных эмбеддингах (gensim)**

In [121]:
def compiling_models(weights, vec_size=100):    
    inputs = tf.keras.layers.Input(shape=(100,))
    embeddings = tf.keras.layers.Embedding(input_dim=len(word2id_1), output_dim=vec_size, 
                                           trainable=True,
                                           weights=[weights])(inputs, )
    mean = tf.keras.layers.Lambda(lambda x: tf.keras.backend.mean(x, axis=1))(embeddings)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(mean)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.Adam()
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [102]:
def get_gensim_weights(model):
    global word2id_1
    weights = np.zeros((len(word2id_1), 100))
    
    for word, i in word2id_1.items():
        if word == 'PAD':
            continue
        try:
            weights[i] = model.wv[word]
        except KeyError:
            weights[i] = model.wv['а']
            
    return weights

In [103]:
w2v_w = get_gensim_weights(w2v)
ft_w = get_gensim_weights(ft)

w2v_model = compiling_models(w2v_w)
ft_model = compiling_models(ft_w)

In [104]:
w2v_model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
          epochs=10)

Epoch 1/10
214/214 [==============================] - 3s 10ms/step - loss: 0.6174 - accuracy: 0.6642 - val_loss: 0.5862 - val_accuracy: 0.6685
Epoch 2/10
214/214 [==============================] - 2s 7ms/step - loss: 0.5750 - accuracy: 0.6705 - val_loss: 0.5490 - val_accuracy: 0.6852
Epoch 3/10
214/214 [==============================] - 2s 7ms/step - loss: 0.5330 - accuracy: 0.7109 - val_loss: 0.5128 - val_accuracy: 0.7393
Epoch 4/10
214/214 [==============================] - 2s 7ms/step - loss: 0.4919 - accuracy: 0.7630 - val_loss: 0.4820 - val_accuracy: 0.7920
Epoch 5/10
214/214 [==============================] - 2s 7ms/step - loss: 0.4567 - accuracy: 0.7972 - val_loss: 0.4580 - val_accuracy: 0.8183
Epoch 6/10
214/214 [==============================] - 1s 7ms/step - loss: 0.4283 - accuracy: 0.8176 - val_loss: 0.4402 - val_accuracy: 0.8239
Epoch 7/10
214/214 [==============================] - 2s 7ms/step - loss: 0.4059 - accuracy: 0.8312 - val_loss: 0.4287 - val_accuracy: 0.8266
Epoch

In [105]:
ft_model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
          epochs=10)

Epoch 1/10
214/214 [==============================] - 3s 10ms/step - loss: 0.6143 - accuracy: 0.6639 - val_loss: 0.5998 - val_accuracy: 0.6685
Epoch 2/10
214/214 [==============================] - 2s 7ms/step - loss: 0.5870 - accuracy: 0.6685 - val_loss: 0.5809 - val_accuracy: 0.6713
Epoch 3/10
214/214 [==============================] - 2s 7ms/step - loss: 0.5631 - accuracy: 0.6823 - val_loss: 0.5685 - val_accuracy: 0.7323
Epoch 4/10
214/214 [==============================] - 2s 7ms/step - loss: 0.5392 - accuracy: 0.7120 - val_loss: 0.5401 - val_accuracy: 0.7129
Epoch 5/10
214/214 [==============================] - 2s 7ms/step - loss: 0.5125 - accuracy: 0.7416 - val_loss: 0.5193 - val_accuracy: 0.7393
Epoch 6/10
214/214 [==============================] - 2s 7ms/step - loss: 0.4864 - accuracy: 0.7707 - val_loss: 0.5000 - val_accuracy: 0.7684
Epoch 7/10
214/214 [==============================] - 2s 7ms/step - loss: 0.4623 - accuracy: 0.7937 - val_loss: 0.4828 - val_accuracy: 0.7878
Epoch

**Учим модели, основанные на предобученных самостоятельно эмбеддингах**

In [113]:
def get_weights(model):
    global word2id, word2id_1
    weights = np.zeros((len(word2id_1), 300))
    
    for word, i in word2id_1.items():
        if word == 'PAD':
            continue
        try:
            weights[i] = model[word2id['университет']]
        except KeyError:
            weights[i] = model[word2id['а']]
            
    return weights

In [122]:
cbow = get_weights(embeddings_cbow)
sg = get_weights(embeddings_sg)

cbow_model = compiling_models(cbow, vec_size=300)
sg_model = compiling_models(sg, vec_size=300)

In [123]:
cbow_model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
          epochs=10)

Epoch 1/10
214/214 [==============================] - 5s 20ms/step - loss: 0.6140 - accuracy: 0.6649 - val_loss: 0.5865 - val_accuracy: 0.6699
Epoch 2/10
214/214 [==============================] - 4s 17ms/step - loss: 0.5526 - accuracy: 0.6908 - val_loss: 0.5131 - val_accuracy: 0.7101
Epoch 3/10
214/214 [==============================] - 4s 17ms/step - loss: 0.4604 - accuracy: 0.7903 - val_loss: 0.4509 - val_accuracy: 0.7975
Epoch 4/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3982 - accuracy: 0.8352 - val_loss: 0.4243 - val_accuracy: 0.8613
Epoch 5/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3653 - accuracy: 0.8509 - val_loss: 0.4107 - val_accuracy: 0.8502
Epoch 6/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3431 - accuracy: 0.8625 - val_loss: 0.4145 - val_accuracy: 0.8641
Epoch 7/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3287 - accuracy: 0.8656 - val_loss: 0.4038 - val_accuracy: 0.8627

In [124]:
sg_model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
          epochs=10)

Epoch 1/10
214/214 [==============================] - 5s 20ms/step - loss: 0.6164 - accuracy: 0.6638 - val_loss: 0.5819 - val_accuracy: 0.6699
Epoch 2/10
214/214 [==============================] - 4s 17ms/step - loss: 0.5432 - accuracy: 0.7002 - val_loss: 0.5038 - val_accuracy: 0.7198
Epoch 3/10
214/214 [==============================] - 4s 17ms/step - loss: 0.4497 - accuracy: 0.7996 - val_loss: 0.4442 - val_accuracy: 0.7961
Epoch 4/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3901 - accuracy: 0.8405 - val_loss: 0.4183 - val_accuracy: 0.8544
Epoch 5/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3591 - accuracy: 0.8562 - val_loss: 0.4078 - val_accuracy: 0.8530
Epoch 6/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3386 - accuracy: 0.8620 - val_loss: 0.4071 - val_accuracy: 0.8613
Epoch 7/10
214/214 [==============================] - 4s 17ms/step - loss: 0.3251 - accuracy: 0.8657 - val_loss: 0.4034 - val_accuracy: 0.8655

**Учим модель с rus-vectores**

In [128]:
ruscor_model = gensim.models.KeyedVectors.load_word2vec_format("ruscorpora_upos_skipgram_300_5_2018.vec")

In [161]:
from pymystem3 import Mystem

m = Mystem()

mapping = {}
for line in open('./ru-rnc.map'):
    ms, ud = line.strip('\n').split()
    mapping[ms] = ud

def get_weights_pretrained_model(model):
    global word2id_1, m, mapping, pos_word_storage
    weights = np.zeros((len(word2id_1), 300))
    norm_words = m.analyze(' '.join(word2id_1.keys()))
    
    for word, i in word2id_1.items():
        if word == 'PAD':
            continue
        pos = 'UNKN'
        if 'analysis' in norm_words[i-1] and len(norm_words[i-1]["analysis"]) > 1:
            pos1 = norm_words[i-1]["analysis"][0]["gr"].split(',')[0]
            if pos1 in mapping:
                pos = mapping[pos1]
        try:
            weights[i] = model[f"{word}_{pos}"]
        except KeyError:
            weights[i] = ruscor_model['штука_NOUN']
            
    return weights

In [162]:
ruscor = get_weights_pretrained_model(ruscor_model)
rc_model = compiling_models(ruscor, vec_size=300)

In [163]:
rc_model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
          batch_size=64,
          epochs=10)

Epoch 1/10
214/214 [==============================] - 3s 13ms/step - loss: 0.6179 - accuracy: 0.6627 - val_loss: 0.5850 - val_accuracy: 0.6699
Epoch 2/10
214/214 [==============================] - 3s 13ms/step - loss: 0.5444 - accuracy: 0.6956 - val_loss: 0.5032 - val_accuracy: 0.7393
Epoch 3/10
214/214 [==============================] - 3s 13ms/step - loss: 0.4494 - accuracy: 0.8000 - val_loss: 0.4442 - val_accuracy: 0.8502
Epoch 4/10
214/214 [==============================] - 3s 13ms/step - loss: 0.3898 - accuracy: 0.8393 - val_loss: 0.4203 - val_accuracy: 0.8599
Epoch 5/10
214/214 [==============================] - 3s 13ms/step - loss: 0.3573 - accuracy: 0.8551 - val_loss: 0.4147 - val_accuracy: 0.8599
Epoch 6/10
214/214 [==============================] - 3s 13ms/step - loss: 0.3379 - accuracy: 0.8644 - val_loss: 0.4053 - val_accuracy: 0.8655
Epoch 7/10
214/214 [==============================] - 3s 13ms/step - loss: 0.3253 - accuracy: 0.8660 - val_loss: 0.4047 - val_accuracy: 0.8613

**Сравним результаты**

In [164]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [194]:
precision_score(y_valid, predicted, average='weighted')

C:\Users\PC Natalya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.4487699123385805

In [195]:
models = {'w2v': w2v_model, 'fasttext': ft_model, 'cbow': cbow_model, 'skipgram': sg_model, 'rus-vectores': rc_model}

def score_generator():
    global models, X_valid, y_valid
    for name, model in models.items():
        predicted = model.predict(X_valid)
        predicted = np.argmax(predicted, axis=1)
        precision = precision_score(y_valid, predicted, average='weighted')
        recall = recall_score(y_valid, predicted, average='weighted')
        f1 = f1_score(y_valid, predicted, average='weighted')
        yield (name, precision, recall, f1)
        
scores = {'precision': [],
          'recall': [],
          'f1': []}
idx = []

for name, precision, recall, f1 in score_generator():
    scores['precision'].append(precision)
    scores['recall'].append(recall)
    scores['f1'].append(f1)
    idx.append(name)

 1/23 [>.............................] - ETA: 0s

C:\Users\PC Natalya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC Natalya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 1/23 [>.............................] - ETA: 0s

C:\Users\PC Natalya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\PC Natalya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


23/23 [==============================] - 0s 3ms/step


C:\Users\PC Natalya\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [196]:
scores_df = pd.DataFrame(scores, index=idx)
scores_df

,precision,recall,f1
w2v,0.44877,0.669903,0.53748
fasttext,0.44877,0.669903,0.53748
cbow,0.44877,0.669903,0.53748
skipgram,0.44877,0.669903,0.53748
rus-vectores,0.44877,0.669903,0.53748


**Результаты работы моделей одинаковы**, поскольку все они предсказывают только 0-вой класс